<a href="https://colab.research.google.com/github/JayYongjaeKim/MoLab/blob/main/%EA%B0%80%EC%9C%84%EB%B0%94%EC%9C%84%EB%B3%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import zipfile

base_dir = '/content/drive/MyDrive/MoLab/data'

zip_files = ['scissor.zip', 'rock.zip', 'paper.zip']

for zip_file in zip_files:
    zip_path = os.path.join(base_dir, zip_file)
    extract_path = os.path.join(base_dir, zip_file.split('.')[0])
    print(f"Extracting {zip_file} to {extract_path}")

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    print(f"{zip_file} 압축 해제 완료!")


Extracting scissor.zip to /content/drive/MyDrive/MoLab/data/scissor
scissor.zip 압축 해제 완료!
Extracting rock.zip to /content/drive/MyDrive/MoLab/data/rock
rock.zip 압축 해제 완료!
Extracting paper.zip to /content/drive/MyDrive/MoLab/data/paper
paper.zip 압축 해제 완료!


In [8]:
import tensorflow as tf
import numpy as np

print(tf.__version__)
print(np.__version__)

2.17.1
1.26.4


In [25]:
import zipfile

base_dir = '/content/drive/MyDrive/MoLab/data'
train_zip_files = ['scissor.zip', 'rock.zip', 'paper.zip']
test_zip_files = ['scissor2.zip', 'rock2.zip', 'paper2.zip'] #2버전에 해당하는 동균님 데이터(노이즈 있음 > 손 흔드는 사진)

print("Training data extraction:")
for zip_file in train_zip_files:
    zip_path = os.path.join(base_dir, zip_file)
    extract_path = os.path.join(base_dir, zip_file.split('.')[0])
    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"{zip_file} 압축 해제 완료")
    else:
        print(f"Error: {zip_file} 파일이 존재하지 않습니다.")

# 테스트 데이터 압축 해제
print("Test data extraction:")
for zip_file in test_zip_files:
    zip_path = os.path.join(base_dir, zip_file)
    extract_path = os.path.join(base_dir, zip_file.split('.')[0])
    if os.path.exists(zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"{zip_file} 압축 해제 완료")
    else:
        print(f"Error: {zip_file} 파일이 존재하지 않습니다.")

Training data extraction:
scissor.zip 압축 해제 완료
rock.zip 압축 해제 완료
paper.zip 압축 해제 완료
Test data extraction:
scissor2.zip 압축 해제 완료
rock2.zip 압축 해제 완료
paper2.zip 압축 해제 완료


In [26]:
import glob
import os
from PIL import Image

def resize_images(img_path):
    images = glob.glob(img_path + "/*.jpg") + glob.glob(img_path + "/*.jpeg") + glob.glob(img_path + "/*.JPG")
    print(len(images), "images to be resized.")

    target_size = (28, 28)
    for img in images:
        old_img = Image.open(img)
        new_img = old_img.resize(target_size, Image.Resampling.LANCZOS)
        new_img.save(img, "JPEG")

    print(len(images), "images resized.")

train_folders = ['scissor', 'rock', 'paper']
test_folders = ['scissor2', 'rock2', 'paper2']

print("Resizing training data:")
for folder in train_folders:
    folder_path = os.path.join(base_dir, folder)
    if os.path.exists(folder_path):
        resize_images(folder_path)

print("Resizing test data:")
for folder in test_folders:
    folder_path = os.path.join(base_dir, folder)
    if os.path.exists(folder_path):
        resize_images(folder_path)

Resizing training data:
100 images to be resized.
100 images resized.
100 images to be resized.
100 images resized.
100 images to be resized.
100 images resized.
Resizing test data:
100 images to be resized.
100 images resized.
101 images to be resized.
101 images resized.
103 images to be resized.
103 images resized.


In [27]:
def load_data(img_path, folders):
    img_size = 28
    color = 3

    total_images = sum([len(glob.glob(os.path.join(img_path, folder, '*.jpg'))) for folder in folders])
    imgs = np.zeros((total_images, img_size, img_size, color), dtype=np.float32)
    labels = np.zeros(total_images, dtype=np.int32)

    idx = 0
    for label, folder in enumerate(folders):
        folder_path = os.path.join(img_path, folder)
        for file in glob.glob(folder_path + "/*.jpg"):
            img = Image.open(file).convert("RGB").resize((img_size, img_size))
            img = np.array(img, dtype=np.float32) / 255.0
            imgs[idx, :, :, :] = img
            labels[idx] = label % 3
            idx += 1

    print("데이터 개수:", idx)
    return imgs, labels

In [28]:
train_folders = ['scissor', 'rock', 'paper']
x_train, y_train = load_data(base_dir, train_folders)

test_folders = ['scissor2', 'rock2', 'paper2']
x_test, y_test = load_data(base_dir, test_folders)

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

데이터 개수: 300
데이터 개수: 304
x_train shape: (300, 28, 28, 3)
y_train shape: (300,)
x_test shape: (304, 28, 28, 3)
y_test shape: (304,)


In [29]:
n_channel_1=16
n_channel_2=32
n_dense=32
n_train_epoch=10

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 26, 26, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 13, 13, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 11, 11, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 5, 5, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 800)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │          25,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 30,819 (120.39 KB)

 Trainable params: 30,819 (120.39 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x_train, y_train, epochs=n_train_epoch)

model.summary()

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9551 - loss: 0.1162
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9942 - loss: 0.0552
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0441
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0223
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 0.0158
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 1.0000 - loss: 0.0135
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 1.0000 - loss: 0.0127
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.0101
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 1.0000 - loss: 0.0090
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 1.0000 - loss: 0.0059


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 26, 26, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 13, 13, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 11, 11, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 5, 5, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 800)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │          25,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 92,459 (361.17 KB)

 Trainable params: 30,819 (120.39 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 61,640 (240.79 KB)

In [34]:
base_dir = "/content/drive/MyDrive/MoLab/data"

print("Resizing training images:")
train_folders = ["scissor", "rock", "paper"]
for folder in train_folders:
    resize_images(os.path.join(base_dir, folder))

print("Resizing test images:")
test_folders = ["scissor2", "rock2", "paper2"]
for folder in test_folders:
    resize_images(os.path.join(base_dir, folder))

print("\nLoading training data:")
x_train, y_train = load_data(base_dir, train_folders)

print("\nLoading test data:")
x_test, y_test = load_data(base_dir, test_folders)

print("\nx_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

Resizing training images:
100 images to be resized.
100 images resized.
100 images to be resized.
100 images resized.
100 images to be resized.
100 images resized.
Resizing test images:
100 images to be resized.
100 images resized.
101 images to be resized.
101 images resized.
103 images to be resized.
103 images resized.

Loading training data:
데이터 개수: 300

Loading test data:
데이터 개수: 304

x_train shape: (300, 28, 28, 3)
y_train shape: (300,)
x_test shape: (304, 28, 28, 3)
y_test shape: (304,)


In [35]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - 35ms/step - accuracy: 0.4868 - loss: 2.8598
test_loss: 2.8597898483276367 
test_accuracy: 0.4868420958518982


# 2차 시도

In [36]:
unique, counts = np.unique(y_train, return_counts=True)
print("Class distribution in training set:", dict(zip(unique, counts)))

Class distribution in training set: {0: 100, 1: 100, 2: 100}


In [42]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 124ms/step - accuracy: 0.3770 - loss: 1.0893 - val_accuracy: 0.4112 - val_loss: 1.0862
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 0.7195 - loss: 0.9063 - val_accuracy: 0.4013 - val_loss: 1.2193
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.7831 - loss: 0.5915 - val_accuracy: 0.4836 - val_loss: 1.4912
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9154 - loss: 0.3397 - val_accuracy: 0.5592 - val_loss: 1.6929
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9524 - loss: 0.1579 - val_accuracy: 0.5230 - val_loss: 1.3983
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.9793 - loss: 0.0873 - val_accuracy: 0.5822 - val_loss: 1.7656
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.9875 - loss: 0.0388 - val_accuracy: 0.5132 - val_loss: 1.6932
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 1.0000 - loss: 0.0187 - val_accuracy: 0.5987 -

In [43]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - 34ms/step - accuracy: 0.6020 - loss: 2.0101
test_loss: 2.010063886642456 
test_accuracy: 0.6019737124443054


# 3차 시도

In [45]:
model = keras.models.Sequential()


model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(3, activation='softmax'))


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.2883 - loss: 1.1262 - val_accuracy: 0.4342 - val_loss: 1.0979
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.3448 - loss: 1.0961 - val_accuracy: 0.3487 - val_loss: 1.0977
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.3495 - loss: 1.0952 - val_accuracy: 0.4178 - val_loss: 1.0952
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.4870 - loss: 1.0445 - val_accuracy: 0.4211 - val_loss: 1.0838
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 154ms/step - accuracy: 0.6446 - loss: 0.8772 - val_accuracy: 0.5493 - val_loss: 1.0649
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.6963 - loss: 0.6640 - val_accuracy: 0.5592 - val_loss: 1.2417
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.7393 - loss: 0.5628 - val_accuracy: 0.5263 - val_loss: 1.6355
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.8451 - loss: 0.3746 - val_accuracy: 0.6349 - 

In [46]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - 19ms/step - accuracy: 0.6414 - loss: 1.7672
test_loss: 1.7671526670455933 
test_accuracy: 0.6414473652839661


# 4차시도

In [51]:
from tensorflow.keras.layers import BatchNormalization, Activation, Dropout

model = keras.models.Sequential()

model.add(keras.layers.Conv2D(32, (3,3), input_shape=(28,28,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(keras.layers.MaxPooling2D((2,2)))

model.add(keras.layers.Conv2D(64, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(keras.layers.MaxPooling2D((2,2)))

model.add(keras.layers.Conv2D(128, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=13, validation_data=(x_test, y_test))

Epoch 1/13
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 93ms/step - accuracy: 0.5710 - loss: 1.0749 - val_accuracy: 0.3388 - val_loss: 1.1016
Epoch 2/13
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.9356 - loss: 0.1870 - val_accuracy: 0.3388 - val_loss: 1.0953
Epoch 3/13
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.9901 - loss: 0.0709 - val_accuracy: 0.3289 - val_loss: 1.1967
Epoch 4/13
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.9974 - loss: 0.0377 - val_accuracy: 0.3289 - val_loss: 1.3480
Epoch 5/13
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - accuracy: 0.9943 - loss: 0.0366 - val_accuracy: 0.3289 - val_loss: 1.4481
Epoch 6/13
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.9942 - loss: 0.0354 - val_accuracy: 0.3289 - val_loss: 1.5265
Epoch 7/13
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 1.0000 - loss: 0.0213 - val_accuracy: 0.3289 - val_loss: 1.6519
Epoch 8/13
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 1.0000 - loss: 0.0177 - val_accuracy: 0.3289 -

In [50]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - 15ms/step - accuracy: 0.3289 - loss: 2.2963
test_loss: 2.2962756156921387 
test_accuracy: 0.32894736528396606


## 3차 시도 64%의 결과가 최고의 결과